In [1]:
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
import torch
from torchvision import transforms

myTransform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

class config:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    lr = 3e-4
    zDim = 128
    imageDim = 28*28*1
    batchSize = 32
    numEpochs = 100
    logstep = 625


In [3]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader

print(f"\nLoading MNIST DS")
dataset = datasets.MNIST(root="dataset/",
                         transform=myTransform,
                         download=True)
loader = DataLoader(dataset,
                    batch_size=config.batchSize,
                    shuffle=True)


Loading MNIST DS


In [4]:
# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, infeatures, hiddenDim=512, lr=0.01):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(infeatures, hiddenDim),
            nn.LeakyReLU(negative_slope=lr),
            nn.Linear(hiddenDim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x)

In [5]:
# Generator class
class Generator(nn.Module):
    def __init__(self, zDim, imageDim, hiddenDim=512):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(zDim, hiddenDim),
            nn.ReLU(),
            nn.Linear(hiddenDim, imageDim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

# Create discriminator and generator instances
discriminator = Discriminator(config.imageDim).to(config.device)
generator = Generator(config.zDim, config.imageDim).to(config.device)

In [6]:
#Fixed Noise
fixedNoise = torch.randn((config.batchSize, config.zDim)).to(config.device)

In [7]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

print(f"\nSetting Optimizers")
optDisc = optim.Adam(discriminator.parameters(), lr=config.lr)
optGen = optim.Adam(generator.parameters(), lr=config.lr)

criterion = nn.BCELoss()
# Initialize SummaryWriters
writerFake = SummaryWriter(f"C:\\Users\\raksh\\OneDrive\\Documents\\DL_Writer\\fake1")
writerReal = SummaryWriter(f"C:\\Users\\raksh\\OneDrive\\Documents\\DL_Writer\\real1")


Setting Optimizers


In [8]:
print(f"\nStarted Training and visualization...")
fixedNoise = torch.randn(config.batchSize, config.zDim).to(config.device)  # Define fixed noise for visualization
step = 0  # Initialize step counter

for epoch in range(config.numEpochs):
    print('-' * 80)
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(config.device)
        batch_size = real.shape[0]
        noise = torch.randn(batch_size, config.zDim).to(config.device)
        fake = generator(noise)
        
        # Train Discriminator
        discReal = discriminator(real).view(-1)
        discFake = discriminator(fake.detach()).view(-1)
        
        lossDreal = criterion(discReal, torch.ones_like(discReal))
        lossDfake = criterion(discFake, torch.zeros_like(discFake))
        
        lossD = (lossDreal + lossDfake) / 2
        discriminator.zero_grad()
        lossD.backward()
        optDisc.step()
        
        # Train Generator
        output = discriminator(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        generator.zero_grad()
        lossG.backward()
        optGen.step()
        
        if batch_idx % config.logstep == 0:
            print(
                f"Epoch [{epoch}/{config.numEpochs}] Batch {batch_idx}/{len(loader)} \
                Loss Disc: {lossD:.4f}, Loss Gen: {lossG:.4f}"
            )
            with torch.no_grad():
                # Generate noise via Generator
                fake = generator(fixedNoise).reshape(-1, 1, 28, 28)
                # Get real data
                data = real.reshape(-1, 1, 28, 28)
                # Plot the grid
                imgGridFake = torchvision.utils.make_grid(fake, normalize=True)
                imgGridReal = torchvision.utils.make_grid(data, normalize=True)

                writerFake.add_image("MNIST Fake Images", imgGridFake, global_step=step)
                writerReal.add_image("MNIST Real Images", imgGridReal, global_step=step)

                writerFake.flush()
                writerReal.flush()

            step += 1  # Increment step counter

writerFake.close()
writerReal.close()



Started Training and visualization...
--------------------------------------------------------------------------------
Epoch [0/100] Batch 0/1875                 Loss Disc: 0.6641, Loss Gen: 0.7090
Epoch [0/100] Batch 625/1875                 Loss Disc: 0.2328, Loss Gen: 1.7007
Epoch [0/100] Batch 1250/1875                 Loss Disc: 0.3253, Loss Gen: 1.5002
--------------------------------------------------------------------------------
Epoch [1/100] Batch 0/1875                 Loss Disc: 0.6990, Loss Gen: 0.9570
Epoch [1/100] Batch 625/1875                 Loss Disc: 0.6054, Loss Gen: 1.4491
Epoch [1/100] Batch 1250/1875                 Loss Disc: 0.5451, Loss Gen: 0.9828
--------------------------------------------------------------------------------
Epoch [2/100] Batch 0/1875                 Loss Disc: 0.5516, Loss Gen: 1.1579
Epoch [2/100] Batch 625/1875                 Loss Disc: 0.3073, Loss Gen: 2.1936
Epoch [2/100] Batch 1250/1875                 Loss Disc: 0.3679, Loss Gen:

# run in Anaconda Prompt
tensorboard --logdir="C:\Users\raksh\OneDrive\Documents\DL_Writer"